## preprocess the linespline data

In [1]:
## import support libraries
import pandas as pd
import re
import os
import unittest

In [13]:
## get the original dimensions of the vessel

file_path = r'C:\Users\907932\Desktop\Software\Diffrac_related\Diffrac database\001_linesplan database\Original linesplan database Journee Versluis\Versluis.003'
with open(file_path) as linesplan_info:
            raw_info = linesplan_info.read().split('\n')
            info = raw_info[1]
            info_crossection = raw_info[3]
            info_pro = info.split(' ')
            print(info_pro)
            vessel_name = info_pro[0][:-1]
            
            try:
                float(info_pro[2])
                vessel_type = info_pro[1]
                vesselLoA = float(info_pro[2])
                vesselBeam = float(info_pro[4])
                vesselDraft = float(info_pro[6])
                vesselDepth = float(info_pro[7][1:-1])
            except:
                vessel_type = info_pro[1] + info_pro[2]
                vesselLoA = float(info_pro[3])
                vesselBeam = float(info_pro[5])
                vesselDraft = float(info_pro[7])
                vesselDepth = float(info_pro[8][1:-1])
            num_cross_sections_pattern = re.compile(r'\d+')
            num_cross_sections_int = int(num_cross_sections_pattern.search(info_crossection).group(0))

            linesplan_info.close()



['VERSLUIS.003:', 'Bulkcarrier,', '187.00', 'x', '29.00', 'x', '10.95', '(13.00)', 'meter.', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [10]:
## get the coordinates of the scatters 
import numpy as np
import tkinter as tk
import linecache
import math
import re

'''
Hanwei Wangd RHDHV 7_2_2019
'''

class linesplan():
    def __init__(self, linsplan_path):
        self.L = None
        self.B = None
        self.T = None  # draft of the ship
        self.D = None  # depth of the ship
        self.name = None
        self.type = None
        self.coordinate = None
        self.num_cross_section = None
        self.fractions = None
        self.scaling_flag = None
        self.kxx = None
        self.kyy = None
        self.KG = None
        self.displacement = None
#         self.file_path = r'C:\Users\907932\Desktop\Software\Diffrac_related\Diffrac database\001_linesplan database\linesplans\coaster_v08.linesplan'
#         self.file_path = r'C:\Users\907932\Desktop\Software\Diffrac_related\Diffrac database\001_linesplan database\Original linesplan database Journee Versluis\Versluis.004'
        self.file_path = linsplan_path

    def get_info(self):
        # get the originald ship dimensions(L, B, T, D)
        with open(self.file_path) as linesplan_info:
            raw_info = linesplan_info.read().split('\n')[1]
            info = raw_info[1]
            info_crossection = raw_info[3]
            info_pro = info.split(' ')
            print(info_pro)
            vessel_name = info_pro[0][:-1]
            
            try:
                float(info_pro[2])
                vessel_type = info_pro[1]
                vesselLoA = float(info_pro[2])
                vesselBeam = float(info_pro[4])
                vesselDraft = float(info_pro[6])
                vesselDepth = float(info_pro[7][1:-1])
            except:
                vessel_type = info_pro[1] + info_pro[2]
                vesselLoA = float(info_pro[3])
                vesselBeam = float(info_pro[5])
                vesselDraft = float(info_pro[7])
                vesselDepth = float(info_pro[8][1:-1])
            num_cross_sections_pattern = re.compile(r'\d+')
            num_cross_sections_int = int(num_cross_sections_pattern.search(info_crossection).group(0))

            linesplan_info.close()
            
            self.L = vesselLoA
            self.B = vesselBeam
            self.T = vesselDraft
            self.D = vesselDepth
            self.name = vessel_name
            self.type = vessel_type
            self.num_cross_section = num_cross_sections_int
        linesplan_info.close()

    def get_fraction(self):

        fractions = []
        fraction_start = 4
        fraction_end = 4 + 1 + math.ceil((self.num_cross_section + 1)/6)
        for i in np.arange(fraction_start+1,fraction_end):
            line = linecache.getline(self.file_path, i)
            for fraction in line.strip('\n').split(' '):
                # print(fraction)
                if fraction != '':
                    fractions.append(float(fraction))
        self.fractions = fractions


    def get_coordinate(self):
        '''

        '''
        # First find the origin of the ship
        # X_0 is the stern of ship, y_0 is central line, z_0 is keel
        fraction_sum = [0]
        for i in range(1, len(self.fractions)+1):   # 千万注意啊
            fraction_sum.append(sum(self.fractions[0: i]))

        # print(' fraction: {}'.format(fraction_sum))
        x_cors = np.array(fraction_sum) * self.L

        with open(self.file_path, 'r') as plan:
             content = plan.read().split('\n')
             Content = []
             effective_content = content[4 + math.ceil((self.num_cross_section)/6) + 1: -3]
             for items in effective_content:
                 for item in items.split(' '):
                     if item != '':
                        Content.append(float(item))
             count = 0
             Coordinates = []

             while Content:
                 index = 1
                 num_points_per_section = int(Content[index]) + 1
                 y_cor = np.array(Content[index + 3: index + num_points_per_section*2 + 3 : 2]) * self.T
                 y_cor = np.sort(y_cor)
                 z_cor = np.array(Content[index + 2: index + num_points_per_section*2 + 2 : 2]) * self.B # !!!!!!!
                 z_cor = np.sort(z_cor)
                 # if count == 0:
                 #     y_cor = np.insert( y_cor, 0, self.T)
                 #     z_cor = np.insert(z_cor,0, 0)
                 Content = Content[index + num_points_per_section*2 + 2: ] # 不是-1, 因为列表不会包含最后一个值
                 # organize coordinate per section
                 try:
                     x_cor = np.ones(len(y_cor)) * x_cors[count]
                     coordinates = list(zip(x_cor, y_cor, z_cor))
                     Coordinates.append(coordinates)
                     count += 1
                 except:
                     print('The ship model is wrong, you may lose the last cross-section!')

             plan.close()
             # Coordinates[0] = np.insert(Coordinates[0], 0, (0 , 0, self.T))
             # Coordinates[-1] = np.append(Coordinates[-1], (self.L, 0, self.T))
             # self.num_cross_section = self.num_cross_section
             self.coordinate = Coordinates

In [11]:
linesplan_processor = linesplan(linsplan_path= file_path)
linesplan_processor.get_info()
linesplan_processor.get_fraction()
linesplan_processor.get_coordinate()

['E']


IndexError: list index out of range

In [ ]:
len(linesplan_processor.fractions)

In [ ]:
linesplan_processor.coordinate
